In [ ]:
# ---- UTILIZANDO a BASE MUSIC_STORE

In [2]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('data/music-store.db')
def consulta(query):
    return pd.read_sql(query, conn)

In [125]:
# Qual artista possui mais albuns? 

consulta("""
         SELECT
            a.name ActorName,
         COUNT(ab.id) as QtyAlbums
         FROM artists a
            INNER JOIN albums ab 
            ON a.id = ab.artist_id 
         GROUP BY artist_id
         ORDER BY QtyAlbums DESC
         LIMIT 1
         """)

,ActorName,QtyAlbums
0,Iron Maiden,21


In [31]:
# Existem clientes de quantos países? 

consulta("SELECT COUNT (DISTINCT country) AS Qty_Contries FROM CUSTOMERS")

,Qty_Contries
0,24


In [123]:
# Consultar quantidades de clientes por países

consulta("""
         SELECT 
            country,
         COUNT(id) as QtyClient 
         FROM customers c
         GROUP BY country
         ORDER BY QtyClient DESC
         """)


,country,QtyClient
0,USA,13
1,Canada,8
2,France,5
3,Brazil,5
4,Germany,4
5,United Kingdom,3
6,Portugal,2
7,India,2
8,Czech Republic,2
9,Sweden,1


In [126]:
consulta("""
         SELECT
            country, 
         COUNT(id) as QtyClient
         FROM customers c
         WHERE country='Brazil'
         GROUP BY country 
         ORDER BY QtyClient DESC
         """)

,country,QtyClient
0,Brazil,5


In [165]:
# Qual genero possui mais músicas?

consulta("""
         SELECT
            g.name genres,
         COUNT(genre_id) as QtyGenres
         FROM tracks t
            INNER JOIN genres g 
            ON t.genre_id = g.id
         GROUP BY genres
         ORDER BY QtyGenres DESC
         LIMIT 1
         """)




,genres,QtyGenres
0,Rock,1297


In [136]:
consulta("""
         SELECT
            g.name genres,
         COUNT(DISTINCT album_id) as QtyAlbums
         FROM tracks t
            INNER JOIN genres g 
            ON t.genre_id = g.id
         GROUP BY genres
         ORDER BY QtyAlbums DESC    
         """)

,genres,QtyAlbums
0,Rock,117
1,Classical,72
2,Latin,39
3,Metal,35
4,Alternative & Punk,23
5,Jazz,13
6,TV Shows,10
7,Blues,7
8,Alternative,5
9,Soundtrack,4


In [163]:
consulta("""
         SELECT
         COUNT(DISTINCT album_id) as QtyAlbums
         FROM tracks t
            INNER JOIN genres g 
            ON t.genre_id = g.id
        WHERE g.name = 'Rock'       
         """)   

,QtyAlbums
0,117


In [178]:
consulta("""
         SELECT
            g.name genres,
         COUNT(DISTINCT a.id) as QtyAlbums
         FROM albums a
            INNER JOIN tracks t 
            ON t.album_id = a.id
            INNER JOIN genres g 
            ON t.genre_id = g.id
            GROUP BY g.name
            ORDER BY QtyAlbums DESC   
         """)

,genres,QtyAlbums
0,Rock,117
1,Classical,72
2,Latin,39
3,Metal,35
4,Alternative & Punk,23
5,Jazz,13
6,TV Shows,10
7,Blues,7
8,Alternative,5
9,Soundtrack,4


In [95]:
# Exiba os 5 clientes que mais fizeram pedidos. Qual é o valor da maior compra?

consulta("""
         SELECT
            customer_id,
            first_name ||' '|| last_name as fullname,
        COUNT(i.id) as QtyOrder,
        SUM(total) as TotalOrder
        FROM invoices i
            INNER JOIN customers c 
            ON i.customer_id = c.id 
        GROUP BY fullname
        ORDER BY QtyOrder DESC, TotalOrder DESC
        LIMIT 5   
         """)

,customer_id,fullname,QtyOrder,TotalOrder
0,6,Helena Holý,7,49.62
1,26,Richard Cunningham,7,47.62
2,57,Luis Rojas,7,46.62
3,46,Hugh O'Reilly,7,45.62
4,45,Ladislav Kovács,7,45.62


In [3]:
# Quais os emails dos 10 clientes que mais gastaram na loja? 

consulta("""
         SELECT
            customer_id,
            first_name ||' '|| last_name as fullname,
            email,
        SUM(total) as TotalPrice
        FROM invoices i
            INNER JOIN customers c 
            ON i.customer_id = c.id 
        GROUP BY fullname
        ORDER BY TotalPrice DESC
        LIMIT 10   
         """)

,customer_id,fullname,email,TotalPrice
0,6,Helena Holý,hholy@gmail.com,49.62
1,26,Richard Cunningham,ricunningham@hotmail.com,47.62
2,57,Luis Rojas,luisrojas@yahoo.cl,46.62
3,45,Ladislav Kovács,ladislav_kovacs@apple.hu,45.62
4,46,Hugh O'Reilly,hughoreilly@apple.ie,45.62
5,28,Julia Barnett,jubarnett@gmail.com,43.62
6,37,Fynn Zimmermann,fzimmermann@yahoo.de,43.62
7,24,Frank Ralston,fralston@gmail.com,43.62
8,25,Victor Stevens,vstevens@yahoo.com,42.62
9,7,Astrid Gruber,astrid.gruber@apple.at,42.62


In [14]:
# Qual o valor médio de cada compra? 

consulta('SELECT AVG(total) AS avg_price FROM invoices')
    

,avg_price
0,5.651942


In [30]:
# Quantas compras estão acima da média?

consulta("""
        SELECT 
        COUNT(total) AS avg_price,
        CASE
            WHEN total > (SELECT AVG(total) AS avg_price FROM invoices) THEN 'AcimaAVG'
            WHEN total < (SELECT AVG(total) AS avg_price FROM invoices) THEN 'abaixoAVG'
        END as categoria
        FROM invoices
        GROUP BY categoria
        """)




,avg_price,categoria
0,179,AcimaAVG
1,233,abaixoAVG


In [81]:
# Vamos fazer uma playlist com as 12 músicas mais vendidas! 

consulta("""SELECT
                t.id trackID,
                t.name MusicName,
            COUNT(track_id) AS QtyTrackSold
            FROM invoice_lines il
                INNER JOIN tracks t
                ON il.track_id = t.id
             GROUP BY trackID
             ORDER BY QtyTrackSold DESC
             LIMIT 12     
            """)



,trackID,MusicName,QtyTrackSold
0,2,Balls to the Wall,2
1,8,Inject The Venom,2
2,9,Snowballed,2
3,20,Overdose,2
4,32,Deuces Are Wild,2
5,48,Not The Doctor,2
6,66,Por Causa De Você,2
7,84,Welcome Home (Sanitarium),2
8,161,Snowblind,2
9,162,Cornucopia,2


In [65]:
consulta("SELECT * FROM invoice_lines WHERE track_id = '66'")

# 1213

,id,invoice_id,track_id,unit_price,quantity
0,17,4,66,0.99,1
1,1736,320,66,0.99,1


In [77]:
# Quais foram elas? Quais os generos e os artistas?

consulta("""SELECT
                t.id trackID,
                t.name MusicName,
                g.name Genre,
                at.name Artist,
            COUNT(track_id) AS QtyTrackSold
            FROM invoice_lines il
                INNER JOIN tracks t
                ON il.track_id = t.id
                INNER JOIN genres g
                ON g.id = t.genre_id
                INNER JOIN albums a
                ON a.id = t.album_id
                INNER JOIN artists at
                ON at.id = a.artist_id
             GROUP BY trackID
             ORDER BY QtyTrackSold DESC
             LIMIT 12     
            """)

,trackID,MusicName,Genre,Artist,QtyTrackSold
0,2,Balls to the Wall,Rock,Accept,2
1,8,Inject The Venom,Rock,AC/DC,2
2,9,Snowballed,Rock,AC/DC,2
3,20,Overdose,Rock,AC/DC,2
4,32,Deuces Are Wild,Rock,Aerosmith,2
5,48,Not The Doctor,Rock,Alanis Morissette,2
6,66,Por Causa De Você,Jazz,Antônio Carlos Jobim,2
7,84,Welcome Home (Sanitarium),Metal,Apocalyptica,2
8,161,Snowblind,Metal,Black Sabbath,2
9,162,Cornucopia,Metal,Black Sabbath,2


In [ ]:
# DESAFIO DO PROF: Precisamos definir 2 países para ações de marketing. 
# Para um, iremos direcionar campanhas dos albums mais caros, 
# nosso foco são os clientes que não se preocupam com o valor do pedido. 
# Para o outro grupo, que gasta menos, precisamos ser mais assertivos, 
# ecomendando albums mais baratos mas com músicas populares. 
# Qual seria sua estratégia? Quais países poderíamos atender? 
# Quais albuns iremos recomendar para cada um dos países? 
# Apresente a query (ou as queries), a tabela (ou as tabelas) 
# e justifique suas escolhas.

In [105]:
consulta("""
         SELECT
            c.country as Nationality,
        COUNT(DISTINCT c.id) as QtyCustomer,
        COUNT(i.id) as QtyInvoices,
        SUM(total) as TotalAmount
        FROM invoices i
            INNER JOIN customers c 
            ON i.customer_id = c.id 
        GROUP BY Nationality
        ORDER BY QtyInvoices DESC, TotalAmount DESC
        LIMIT 20   
         """)

,Nationality,QtyCustomer,QtyInvoices,TotalAmount
0,USA,13,91,523.06
1,Canada,8,56,303.96
2,France,5,35,195.10
3,Brazil,5,35,190.10
4,Germany,4,28,156.48
5,United Kingdom,3,21,112.86
6,Czech Republic,2,14,90.24
7,Portugal,2,14,77.24
8,India,2,13,75.26
9,Chile,1,7,46.62
